In [ ]:
import tensorflow as tf
from tensorflow import keras

DATASET_DIR= r'C:/Users/Gourab Paul/CollegeProject/tomato'
EPOCHS = 50
BATCH_SIZE = 32
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
IMAGE_CHANNELS = 3
IMAGE_SHAPE=(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)
NUM_CLASSES=10

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    F"{DATASET_DIR}/train",
    validation_split=0.2,
    subset="training",
    # image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    seed=42,
)

val_ds = keras.utils.image_dataset_from_directory(
    F"{DATASET_DIR}/train",
    validation_split=0.2,
    subset="validation",
    # image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    seed=42,
)

In [ ]:
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal_and_vertical"),
    keras.layers.RandomRotation(0.2),
    keras.layers.RandomTranslation(0.2, 0.2),
    keras.layers.RandomZoom(0.2),
    # keras.layers.RandomBrightness(0.4),
    keras.layers.RandomContrast(0.4),
])

In [ ]:
preprocessing = keras.Sequential([
    # keras.layers.Reshape(IMAGE_SHAPE),
    keras.layers.Resizing(IMAGE_HEIGHT, IMAGE_WIDTH),
    keras.layers.Rescaling(1./255),
])

In [ ]:
# AUTOTUNE = tf.data.AUTOTUNE
# train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
# train_ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)

train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

In [ ]:
model = keras.models.Sequential([
    keras.layers.Input(shape=(None,None,3)),

    preprocessing,
    # data_augmentation,

    keras.layers.Conv2D(32, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(),

    keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(),
    
    keras.layers.Conv2D(128, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(),
    
    keras.layers.GlobalAveragePooling2D(),
    
    # tf.keras.layers.Flatten(),

    # keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.2),
    # keras.layers.Dense(128, activation="relu"),
    # keras.layers.Dropout(0.4),

    keras.layers.Dense(NUM_CLASSES, activation="softmax")
])

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

model.summary(expand_nested=True)

In [ ]:
earlystop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=300,
    # steps_per_epoch=train_ds.samples/train_ds.batch_size,
    batch_size=BATCH_SIZE,
    callbacks=[earlystop]
    # validation_steps=validation_ds.samples/validation_ds.batch_size
)

In [ ]:
test_ds = keras.utils.image_dataset_from_directory(
    F"{DATASET_DIR}/val",
    # image_size=(IMAGE_HEIGHT, IMAGE_WIDTH)
)
results = model.evaluate(test_ds, verbose=0)
print("test accuracy:", results[1]*100)
print("test loss:", results[0]*100)

In [ ]:
from matplotlib import pyplot as plt

acc = history.history["sparse_categorical_accuracy"]
val_acc = history.history["val_sparse_categorical_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)

# Train and validation accuracy
plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
# plt.ylim((0, 1))
plt.plot(epochs, acc, "b",label="Training accurarcy")
plt.plot(epochs, val_acc, "r", label="Validation accurarcy")
plt.title("Training and Validation accurarcy")
plt.legend()

# Train and validation loss
plt.subplot(1, 2, 2)

plt.plot(epochs, loss, "b", label="Training loss")
plt.plot(epochs, val_loss, "r", label="Validation loss")
plt.title("Training and Validation loss")
plt.legend()
plt.show()

In [ ]:
import numpy as np
import tensorflow_datasets as tfds

correct = 0
inCorrect = 0
classNames = [
    "Tomato___Bacterial_spot", 
    "Tomato___Early_blight",
    "Tomato___healthy",
    "Tomato___Late_blight",
    "Tomato___Leaf_Mold",
    "Tomato___Septoria_leaf_spot",
    "Tomato___Spider_mites Two-spotted_spider_mite",
    "Tomato___Target_Spot",
    "Tomato___Tomato_mosaic_virus",
    "Tomato___Tomato_Yellow_Leaf_Curl_Virus",
]

def predict(image, label):
    result = model.predict(image)*100
    predictedClass = classNames[result.argmax()]

    boolResult = False
    if str(classNames[label]) == predictedClass:
        boolResult = True
    # print(f"\n{classNames[label]} --- Label {label} --- Result {predictedClass} --- Correct?? {boolResult}")
    return boolResult

valDS = keras.utils.image_dataset_from_directory(
    F"{DATASET_DIR}/val",
    # image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=1,
    shuffle=False,
)

for image, label in valDS:
    result = predict(image, label[0])
    if(result==True):
        correct += 1
    else:
        inCorrect += 1
    # break
print(f"\n[INFO] Correct ---> {correct}           InCorrect ---> {inCorrect}")

In [ ]:
# model.save('tomato_feel_bad(32,64,128_GA,D20)_E103.h5')